In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import json
url = 'http://www.moa.gov.cn/ztzl/zgnmfsj/xwzx/201809/t20180913_6157258.htm'

In [2]:
r = requests.get(url)
r.encoding = r.apparent_encoding

In [3]:
prov= None
meal= None
meal_img_dict = {}
meal_df = pd.DataFrame(columns = ['prov','meal'])
soup = BeautifulSoup(r.text)

for tag in soup.find_all('p'):
    if tag.find_all('strong'):
        prov = tag.text     
    else:
        try: 
            if tag['class']:
                meal_list = re.findall(re.compile(r'[\d]+.[\w]+'),tag.text)
                img_list = tag.find_all('img')
                if len(meal_list)>0:
                    meal = meal_list[0]
                    meal_df = pd.concat([meal_df, pd.DataFrame([[prov,meal]],columns = ['prov','meal'])])
                if len(img_list)>0:
                    meal_img_dict[meal] = img_list[0]['src']    
        except Exception:
            continue

In [4]:
meal_df['prov'] = meal_df['prov'].str.strip('\u3000')
meal_df['code'] = meal_df['prov'].str[0:2]
meal_df = meal_df[meal_df['meal']!='05039419号']
meal_df['meal'] = pd.DataFrame(meal_df['meal'].str.split('、').to_list())[1].values
meal_df.tail()

,prov,meal,code
0,海南省,东山羊,海南
0,海南省,抱罗粉,海南
0,江苏省,羊方藏鱼,江苏
0,江苏省,盐水鸭,江苏
0,江苏省,松鼠鳜鱼,江苏


In [5]:
meal_img_dict.keys()

dict_keys(['1、爆肚', '2、卤煮火烧', '3、老北京炸酱面', '4、老爆三', '5、贴饽饽熬小鱼', '6、狗不理包子', '7、蟹壳黄', '8、响油鳝糊', '9、鸡肉生煎馒头', '10、重庆小面', '11、重庆火锅', '12、重庆辣子鸡', '13、酸辣粉', '14、手扒肉', '15、涮羊肉', '16、烤全羊', '17、蒙古血肠', '18、烤羊肉串', '19、大盘鸡', '20、抓饭', '21、羊杂碎', '22、羊肉手抓', '23、辣爆羊羔肉', '24、芽豆藏香猪', '25、鲁朗石锅鸡', '26、风干牛羊肉', '27、烤乳猪', '28、五色糯米饭', '29、老友粉', '30、得莫利炖活鱼', '31、哈尔滨红肠', '32、杀猪菜', '33、鲶鱼烧茄子', '34、酸菜猪肉炖粉条', '35、熏肉大饼', '36、咸鱼饼子', '37、沟帮子熏鸡', '38、大连', '39、驴肉火烧', '40、万字扣肉', '41、御土荷叶鸡', '42、烩面', '43、胡辣汤', '44、鲤鱼焙面', '45、济南九转大肠', '46、把子肉', '47、淄博博山豆腐箱', '48、枣庄辣子鸡', '49、刀削面', '50、平遥牛肉', '51、过油肉', '52、羊肉泡馍', '53、腊汁肉夹馍', '54、安康蒸面', '55、岐山臊子面', '56、兰州清汤牛肉面', '57、百花全鸡', '58、酿皮子', '59、担担面', '60、回锅肉', '61、麻婆豆腐', '62、钵钵鸡', '63、互助葱花土鸡', '64、麻食', '65、口味虾', '66、剁椒鱼头', '67、湘西三下锅', '68、排骨藕汤', '69、清蒸武昌鱼', '70、红菜苔炒腊肉', '72、瓦罐汤', '73、鳅鱼钻豆腐', '74、三杯鸡', '75、臭鳜鱼', '76、小龙虾', '77、淮南牛肉汤', '78、一品锅', '79、咸菜大汤黄鱼', '80、黄鱼鲞烤猪肉', '81、竹笋炖排骨', '82、豆腐饺', '83、姜母鸭', '84、海蛎煎', '85、白切鸡', '86、潮汕牛肉丸', '87、荔湾艇仔粥', '88、横岗腊鸭', '89、稻草排骨', '90、酸汤鱼', '91、丝娃娃', '

In [6]:
file = '../data/china_cities.csv'
all_cities_df = pd.read_csv(file,encoding='gb18030')
main_cities = all_cities_df[all_cities_df.sort_values(by='行政代码')['行政代码'].astype('str').str.find('0100')>0]
main_cities.columns = ['id','name','lon','lat']
main_cities['code'] = main_cities['name'].str[0:2]
meal_city_df = pd.merge(left = meal_df,right=main_cities,on='code',how='left')
meal_city_df.head()

C:\Users\renb\AppData\Local\Temp\ipykernel_22856\3118300717.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  main_cities = all_cities_df[all_cities_df.sort_values(by='行政代码')['行政代码'].astype('str').str.find('0100')>0]
C:\Users\renb\AppData\Local\Temp\ipykernel_22856\3118300717.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_cities['code'] = main_cities['name'].str[0:2]


,prov,meal,code,id,name,lon,lat
0,北京市,爆肚,北京,110100,北京市,116.395645,39.929986
1,北京市,卤煮火烧,北京,110100,北京市,116.395645,39.929986
2,北京市,老北京炸酱面,北京,110100,北京市,116.395645,39.929986
3,天津市,老爆三,天津,120100,天津市,117.210813,39.143930
4,天津市,贴饽饽熬小鱼,天津,120100,天津市,117.210813,39.143930


In [9]:
meal_city_df_count = meal_city_df.groupby('code').agg({'name':'first','lon':'first','lat':'first','id':'count'})
meal_list  = meal_city_df.groupby('name')['meal'].apply(list).to_frame().reset_index()
meal_list.columns = ['name','meal_list']
meal_list['meal_list'] = meal_list['meal_list'].str.join(",")
meal_city_df_count = pd.merge(left = meal_city_df_count,right = meal_list,on='name')
meal_city_df_count.tail()

,name,lon,lat,id,meal_list
26,辽宁省沈阳市,123.432791,41.808645,3,"咸鱼饼子,沟帮子熏鸡,大连"
27,重庆市,106.530635,29.544606,4,"重庆小面,重庆火锅,重庆辣子鸡,酸辣粉"
28,陕西省西安市,108.953098,34.277800,4,"羊肉泡馍,腊汁肉夹馍,安康蒸面,岐山臊子面"
29,青海省西宁市,101.767921,36.640739,2,"互助葱花土鸡,麻食"
30,黑龙江省哈尔滨市,126.657717,45.773225,3,"得莫利炖活鱼,哈尔滨红肠,杀猪菜"


In [11]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = 'colab'
fig = go.Figure()
fig.add_trace(
go.Scattermapbox(mode='markers+text', 
                                 lon = meal_city_df_count['lon'],
                                 lat = meal_city_df_count['lat'],
                                 hovertext = meal_city_df_count[['name','meal_list']],
                                 hoverinfo = 'text',
                 text = meal_city_df_count['meal_list'].tolist(),
                                 marker=go.scattermapbox.Marker(
           size=meal_city_df_count['id']*10,color='orange',
        ),
                                 )
)
fig.update_layout(mapbox_zoom=4,mapbox_accesstoken=map_box_token,mapbox_style="streets",showlegend=False,height=1200,width=1600,mapbox_center = dict(lat=34,lon=108)
    )
fig.show()

In [12]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians
def geo_distance(p1,p2):
    dis = haversine_distances([[radians(_) for _ in p1], [radians(_) for _ in p2]])[0][1]* 6371000/1000  # multiply by Earth radius to get kilometers
    return dis
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

meal_city_df_count['loc'] = meal_city_df_count.apply(lambda x: list([x['lon'],
                                        x['lat']]),axis=1)  
xv,yv= np.meshgrid(meal_city_df_count['loc'], meal_city_df_count['loc'],indexing='ij')
data = {}
data['distance_matrix'] = [[round(geo_distance(xv[i][j],yv[i][j])) for i in range(xv.shape[0])] for j in range(xv.shape[1])] 
data['num_vehicles'] = 1
data['depot'] = 2

In [13]:
def print_solution(manager, local_routing, solution):
    """Prints solution on console."""
    print('Objective: {} km'.format(solution.ObjectiveValue()))
    index = local_routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not local_routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(local_routing.NextVar(index))
        route_distance += local_routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}km\n'.format(route_distance)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model.
my_routing = pywrapcp.RoutingModel(manager)
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]
transit_callback_index = my_routing.RegisterTransitCallback(distance_callback)
# Define cost of each arc.
my_routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = my_routing.SolveWithParameters(search_parameters)
# Print solution on console.
if solution:
    print_solution(manager, my_routing, solution)

Objective: 10954 km
Route for vehicle 0:
 2 -> 10 -> 17 -> 16 -> 3 -> 6 -> 9 -> 8 -> 14 -> 26 -> 4 -> 30 -> 0 -> 18 -> 23 -> 15 -> 20 -> 21 -> 11 -> 19 -> 12 -> 25 -> 27 -> 5 -> 1 -> 13 -> 24 -> 29 -> 22 -> 7 -> 28 -> 2



In [14]:
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes
routes = get_routes(solution, my_routing, manager)

In [15]:
routes_df = pd.Series(routes[0][:-1]).to_frame()
meal_city_df_count['seq'] = routes_df.reset_index().sort_values(by=0)['index']

In [18]:
import shapely
from shapely.geometry import LineString
import plotly.express as px
linestring = LineString([meal_city_df_count.iloc[routes[0][i],1:3].tolist() for i in range(32)])


In [17]:
lats = []
lons = []
names = []
colors = []

x, y = linestring.xy
lats = np.append(lats, y)
lons = np.append(lons, x)
lats = np.append(lats, None)
lons = np.append(lons, None)
names = np.append(names, None)

fig=  go.Figure()


trace1 = go.Scattermapbox(mode='markers', 
                                 lon = meal_city_df_count['lon'],
                                 lat = meal_city_df_count['lat'],
                                 hovertext = meal_city_df_count['name'],
                                 hoverinfo = 'text',
                                 marker=go.scattermapbox.Marker(
           size=meal_city_df_count['id']*10,color='orange'
        ))
trace2 = go.Scattermapbox(mode='markers+lines+text',lat=lats, lon=lons,
                         text = meal_city_df_count['seq'].tolist(),
                          textfont=dict(family="sans serif",size=18,color='lime'),
                          line=dict(color='darksalmon')
        )
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    legend=dict(bgcolor='black',
    font = dict(size=12,color='silver'),
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01),mapbox_zoom=4,mapbox_accesstoken=map_box_token,mapbox_style="streets",showlegend=False,height=1200,width=1600,mapbox_center = dict(lat=34,lon=108))
fig.show()